# Simple ETL - Graflow Minimal Example

A minimal Extract → Transform → Load workflow.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/GraflowAI/graflow-examples/blob/main/examples/notebooks/simple_etl.ipynb)

## Prerequisites

Install Graflow and import the core modules:
- **`@task`** — Decorator to define a workflow task
- **`workflow`** — Context manager to build and execute a task graph
- **`TaskExecutionContext`** — Provides access to the shared channel for inter-task communication

In [ ]:
!pip install graflow

In [ ]:
from graflow.core.context import TaskExecutionContext
from graflow.core.decorators import task
from graflow.core.workflow import workflow

## Task Definitions

Define three tasks that form a sequential pipeline using `>>` (the sequential composition operator):

```
extract >> transform >> load
```

Each task uses `inject_context=True` to access the shared **channel** — a key-value store for passing data between tasks.

In [ ]:
@task(inject_context=True)
def extract(ctx: TaskExecutionContext):
    """Load raw data."""
    raw_data = [
        {"name": "Alice", "score": 85},
        {"name": "Bob", "score": 42},
        {"name": "Charlie", "score": 73},
        {"name": "Diana", "score": 91},
    ]
    print(f"[Extract] {len(raw_data)} records loaded")
    ctx.get_channel().set("raw_data", raw_data)


@task(inject_context=True)
def transform(ctx: TaskExecutionContext):
    """Filter records with score >= 50 and assign a grade."""
    channel = ctx.get_channel()
    raw = channel.get("raw_data")

    results = []
    for r in raw:
        if r["score"] >= 50:
            grade = "A" if r["score"] >= 90 else "B" if r["score"] >= 70 else "C"
            results.append({**r, "grade": grade})

    print(f"[Transform] {len(raw)} -> {len(results)} records")
    channel.set("transformed", results)


@task(inject_context=True)
def load(ctx: TaskExecutionContext):
    """Output the results."""
    data = ctx.get_channel().get("transformed")
    print("[Load] Results:")
    for r in data:
        print(f"  {r['name']}: {r['score']} ({r['grade']})")

## Run Workflow

Use the `workflow()` context manager to register the task graph and execute it. The `>>` operator wires tasks sequentially, and `wf.execute()` starts execution from the given entry task.

In [ ]:
with workflow("hello_etl") as wf:
    extract >> transform >> load
    wf.execute("extract")